In [24]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np
from Preprocess import Preprocess
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Bidirectional, TimeDistributed, Embedding, Dropout
from tensorflow.keras.optimizers import Adam

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [2]:
# 학습용 데이터 로드
ner = pd.read_csv('../data/수정데이터/명사들.csv', index_col=0)
ner.dropna(inplace=True)
print(ner.head())
p = Preprocess(word2index_dic='../data/chatbot_dict.bin',
               userdic=None)

   nouns label
0  반갑습니다     O
1     소속     O
2     상담     O
3      사     O
4     이름     O


In [31]:
ner.isnull().any()

nouns    False
label    False
dtype: bool

In [40]:
ner[ner['nouns'].str.len() == 0]

,nouns,label


In [3]:
# BIO태그별 각각의 값의 개수 카운트'
print(ner.groupby('label').size().reset_index(name='count'))

  label  count
0   B_A    116
1   B_C     66
2  B_DT     47
3   B_P     31
4   B_R    351
5   B_T     60
6     I     31
7     O  59576
8     X      6


In [4]:
# Word 열의 중복을 제거한 값의 개수
print('nouns 열의 중복을 제거한 값의 개수 : {}'.format(ner['nouns'].nunique()))
print(ner.shape[0])

nouns 열의 중복을 제거한 값의 개수 : 60282
60284


In [5]:
# 말뭉치 데이터에서 단어와 BIO태그만 불러와 학습용 데이터셋 생성
words, tags = ner['nouns'], ner['label']

print('샘플 개수:', len(words))
print('0번 째 샘플 단어 시퀀스:', words[0])
print('0번 째 샘플 bio 태그:', tags[0])
print('샘플 단어 시퀀스 최대 길이:', max(len(l) for l in words))
print('샘플 단어 시퀀스 평균 길이:', (sum(map(len, words)) / len(words)))

샘플 개수: 60284
0번 째 샘플 단어 시퀀스: 반갑습니다
0번 째 샘플 bio 태그: O
샘플 단어 시퀀스 최대 길이: 50
샘플 단어 시퀀스 평균 길이: 3.8354289695441577


In [6]:
# 토크나이저 정의
tag_tokenizer = preprocessing.text.Tokenizer(lower=False)
tag_tokenizer.fit_on_texts(tags)

In [7]:
# 단어사전 및 태그 사전 크기
vocab_size = len(p.word_index) + 1
tag_size = len(tag_tokenizer.word_index) + 1
print('BIO 태그 사전 크기:', tag_size)
print('단어 사전 크기:', vocab_size)

BIO 태그 사전 크기: 11
단어 사전 크기: 91132


In [8]:
# 학습용 단어 시퀀스 생성
x_train = [p.get_wordidx_sequence(word) for word in words]
y_train = tag_tokenizer.texts_to_sequences(tags)
index_to_ner = tag_tokenizer.index_word # 시퀀스 인덱스를 NER로 변환 하기 위해 사용
index_to_ner[0] = 'PAD'

In [9]:
# 시퀀스 패딩 처리
max_len = 50
x_train = preprocessing.sequence.pad_sequences(x_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

In [10]:
# 학습 데이터와 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=10)

In [11]:
# 출력 데이터 원핫인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

print('학습 샘플 시퀀스 형상:', x_train.shape)
print('학습 샘플 레이블 형상:', y_train.shape)
print('테스트 샘플 시퀀스 형상:', x_test.shape)
print('테스트 샘플 레이블 형상:', y_test.shape)

학습 샘플 시퀀스 형상: (48227, 50)
학습 샘플 레이블 형상: (48227, 50, 11)
테스트 샘플 시퀀스 형상: (12057, 50)
테스트 샘플 레이블 형상: (12057, 50, 11)


In [12]:
# 모델 정의 (Bi-LSTM)
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(256, return_sequences=True, dropout=0.50, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10)
model.save('./model/ner_model.h5')

Epoch 1/10
377/377 [==============================] - 305s 782ms/step - loss: 0.0748 - accuracy: 0.7904
Epoch 2/10
377/377 [==============================] - 327s 868ms/step - loss: 0.0108 - accuracy: 0.9728
Epoch 3/10
377/377 [==============================] - 314s 834ms/step - loss: 0.0054 - accuracy: 0.9883
Epoch 4/10
377/377 [==============================] - 308s 816ms/step - loss: 0.0042 - accuracy: 0.9898
Epoch 5/10
377/377 [==============================] - 308s 817ms/step - loss: 0.0037 - accuracy: 0.9905
Epoch 6/10
377/377 [==============================] - 316s 837ms/step - loss: 0.0035 - accuracy: 0.9911
Epoch 7/10
377/377 [==============================] - 315s 835ms/step - loss: 0.0033 - accuracy: 0.9921
Epoch 8/10
377/377 [==============================] - 313s 829ms/step - loss: 0.0031 - accuracy: 0.9931
Epoch 9/10
377/377 [==============================] - 309s 818ms/step - loss: 0.0030 - accuracy: 0.9938
Epoch 10/10
377/377 [==============================] - 304s 805m

In [13]:
# 모델 평가
from tensorflow import keras
model = keras.models.load_model('./model/ner_model.h5')
print('평가 결과:', model.evaluate(x_test, y_test)[1])

377/377 [==============================] - 41s 107ms/step - loss: 0.0033 - accuracy: 0.9930
평가 결과: 0.9929937124252319


In [16]:
# 시퀀스를 NER 태그로 변환하는 함수
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace('PAD', 'O'))
        result.append(temp)
    return result

In [22]:
# f1 스코어
from seqeval.metrics import f1_score, classification_report
y_predicted = model.predict([x_test]) # 테스트 데이터셋의 NER 예측
pred_tags = sequences_to_tag(y_predicted) # 예측된 NER
test_tags = sequences_to_tag(y_test) # 실제 NER
print(classification_report(test_tags, pred_tags))
print('F1-score: {:.1%}'.format(f1_score(test_tags, pred_tags)))

377/377 [==============================] - 41s 109ms/step


c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: R seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: C seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: T seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
c:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: 

              precision    recall  f1-score   support

           T       0.00      0.00      0.00         9
           _       0.00      0.00      0.00        14

   micro avg       0.00      0.00      0.00        23
   macro avg       0.00      0.00      0.00        23
weighted avg       0.00      0.00      0.00        23

F1-score: 0.0%


In [ ]:
# # 모델 정의
# from keras_crf import CRFModel

# embedding_dim = 128
# hidden_units = 64
# dropout_ratio = 0.3
# adam = Adam(learning_rate=0.001)

# sequence_input = Input(shape=(max_len, ), dtype=tf.int32, name='sequence_input')
# model_embedding = Embedding(input_dim=vocab_size,
#                             output_dim=embedding_dim,
#                             input_length=max_len)(sequence_input)
# model_bilstm = Bidirectional(LSTM(hidden_units, return_sequences=True))
# model_dropout = TimeDistributed(Dropout(dropout_ratio))(model_bilstm)
# model_dense = TimeDistributed(Dense(tag_size, activation='relu'))(model_dropout)
# base = Model(inputs=sequence_input, outputs=model_dense)
# model = CRFModel(base, tag_size)
# model.compile(optimizer=adam, metrics='accuracy')

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
# mc = ModelCheckpoint('bilstm_crf/cp.ckpt', monitor='val_decode_sequence_accuracy', mode='max', verbose=1, save_best_only=True, save_weights_only=True)

# history = model.fit(x_train, y_train.astype(int), batch_size=128, epochs=10, callbacks=[mc, es])